<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
import torchaudio
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
# train_dataset = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)

print(valid_dataset)

# # Example: Load the first audio file in the dataset
first_sample = valid_dataset[0]
dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

labels = valid_dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

def edit_exam(examples):
    for i, label in enumerate(examples):
        label2id[label] = str(i)
        id2label[str(i)] = label
        # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
        for i, x in enumerate(examples['file']):
            a = examples['label'][i]
            if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
            elif id2label[str(examples['label'][i])] == '_silence_':
                examples['label'][i] = dict_label['silence']
    return examples



valid_dataset = valid_dataset.map(edit_exam, batched=True)
df = valid_dataset.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# Hiển thị đồ thị
label_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Số lượng')
plt.title('Số lượng mỗi class trong cột label')
plt.show()
# valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
# first_sample

In [ ]:
from transformers import AutoFeatureExtractor
from datasets import load_dataset, Audio
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")


import pandas as pd
import matplotlib.pyplot as plt

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000, truncation=True
    )
    return inputs




def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

feature_extractor = AutoFeatureExtractor.from_pretrained('./model_1')
train_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)


labels = valid_dataset_.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


train_dataset_ = train_dataset_.map(edit_label, batched=True)
valid_dataset_ = valid_dataset_.map(edit_label, batched=True)


df = train_dataset_.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# Hiển thị đồ thị
label_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Số lượng')
plt.title('Số lượng mỗi class trong cột label')
plt.show()

train_dataset_ = train_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
valid_dataset_ = valid_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])

labels = dict_label.keys()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)



train_data = train_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
valid_data = valid_dataset_.cast_column("audio", Audio(sampling_rate=16_000))

encoded_data_train = train_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_data_validation = valid_data.map(preprocess_function, remove_columns="audio", batched=True)
print(encoded_data_train)

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "./model_1", num_labels=num_labels, label2id=label2id, id2label=id2label
)
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_train.with_format("torch"),
    eval_dataset=encoded_data_validation.with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
2024-08-24 11:37:20.574841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2024-08-24 11:37:20.578933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
10    32550
8      1885
6      1864
9      1861
0      1860
1      1853
5      1852
2      1843
3      1842
4      1839
7      1839
11        5
Name: label, dtype: int64
{'yes': '0', 'no': '1', 'up': '2', 'down': '3', 'left': '4', 'right': '5', 'on': '6', 'off': '7', 'stop': '8', 'go': '9', 'unknown': '10', 'silence': '11'}
Dataset({
    features: ['label', 'input_values'],
    num_rows: 51093
})
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ./model_1 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
{'loss': 2.4806, 'grad_norm': 1.4397743940353394, 'learning_rate': 7.518796992481203e-07, 'epoch': 0.03}
{'loss': 2.475, 'grad_norm': 1.3291702270507812, 'learning_rate': 1.5037593984962406e-06, 'epoch': 0.05}
{'loss': 2.468, 'grad_norm': 1.4363396167755127, 'learning_rate': 2.255639097744361e-06, 'epoch': 0.08}
{'loss': 2.4558, 'grad_norm': 1.5088651180267334, 'learning_rate': 3.007518796992481e-06, 'epoch': 0.1}
{'loss': 2.4344, 'grad_norm': 1.6106393337249756, 'learning_rate': 3.7593984962406014e-06, 'epoch': 0.13}
{'loss': 2.3959, 'grad_norm': 2.2357585430145264, 'learning_rate': 4.511278195488722e-06, 'epoch': 0.15}
{'loss': 2.3395, 'grad_norm': 3.230968475341797, 'learning_rate': 5.263157894736842e-06, 'epoch': 0.18}
{'loss': 2.2709, 'grad_norm': 2.9282634258270264, 'learning_rate': 6.015037593984962e-06, 'epoch': 0.2}
{'loss': 2.1967, 'grad_norm': 3.357541084289551, 'learning_rate': 6.766917293233083e-06, 'epoch': 0.23}
{'loss': 2.1455, 'grad_norm': 3.9185028076171875, 'learning_rate': 7.518796992481203e-06, 'epoch': 0.25}
{'loss': 2.0915, 'grad_norm': 3.346215009689331, 'learning_rate': 8.270676691729324e-06, 'epoch': 0.28}
{'loss': 2.0103, 'grad_norm': 2.958008289337158, 'learning_rate': 9.022556390977444e-06, 'epoch': 0.3}
{'loss': 1.9065, 'grad_norm': 3.006120443344116, 'learning_rate': 9.774436090225564e-06, 'epoch': 0.33}
{'loss': 1.8446, 'grad_norm': 2.708524465560913, 'learning_rate': 1.0526315789473684e-05, 'epoch': 0.35}
{'loss': 1.8077, 'grad_norm': 2.4971444606781006, 'learning_rate': 1.1278195488721805e-05, 'epoch': 0.38}
{'loss': 1.7115, 'grad_norm': 3.0925276279449463, 'learning_rate': 1.2030075187969925e-05, 'epoch': 0.4}
{'loss': 1.7005, 'grad_norm': 2.031919240951538, 'learning_rate': 1.2781954887218045e-05, 'epoch': 0.43}
{'loss': 1.5418, 'grad_norm': 2.012681245803833, 'learning_rate': 1.3533834586466165e-05, 'epoch': 0.45}
{'loss': 1.6082, 'grad_norm': 2.0470175743103027, 'learning_rate': 1.4285714285714285e-05, 'epoch': 0.48}
{'loss': 1.5561, 'grad_norm': 1.292642593383789, 'learning_rate': 1.5037593984962406e-05, 'epoch': 0.5}
{'loss': 1.5561, 'grad_norm': 1.0164387226104736, 'learning_rate': 1.5789473684210526e-05, 'epoch': 0.53}
{'loss': 1.5481, 'grad_norm': 0.8144148588180542, 'learning_rate': 1.6541353383458648e-05, 'epoch': 0.55}
{'loss': 1.5258, 'grad_norm': 0.9964609742164612, 'learning_rate': 1.7293233082706766e-05, 'epoch': 0.58}
{'loss': 1.5403, 'grad_norm': 1.9777063131332397, 'learning_rate': 1.8045112781954888e-05, 'epoch': 0.6}
{'loss': 1.5077, 'grad_norm': 0.5350005030632019, 'learning_rate': 1.8796992481203007e-05, 'epoch': 0.63}
{'loss': 1.5258, 'grad_norm': 3.122205972671509, 'learning_rate': 1.954887218045113e-05, 'epoch': 0.65}
{'loss': 1.4499, 'grad_norm': 0.7446007132530212, 'learning_rate': 2.0300751879699247e-05, 'epoch': 0.68}
{'loss': 1.5157, 'grad_norm': 0.9741470813751221, 'learning_rate': 2.105263157894737e-05, 'epoch': 0.7}
{'loss': 1.5031, 'grad_norm': 1.0316869020462036, 'learning_rate': 2.1804511278195487e-05, 'epoch': 0.73}
{'loss': 1.5592, 'grad_norm': 0.821504533290863, 'learning_rate': 2.255639097744361e-05, 'epoch': 0.75}
{'loss': 1.5338, 'grad_norm': 0.7279519438743591, 'learning_rate': 2.3308270676691728e-05, 'epoch': 0.78}
{'loss': 1.5007, 'grad_norm': 1.4839861392974854, 'learning_rate': 2.406015037593985e-05, 'epoch': 0.8}
{'loss': 1.4984, 'grad_norm': 1.9492813348770142, 'learning_rate': 2.4812030075187968e-05, 'epoch': 0.83}
{'loss': 1.5633, 'grad_norm': 1.9851725101470947, 'learning_rate': 2.556390977443609e-05, 'epoch': 0.85}
{'loss': 1.4193, 'grad_norm': 1.8006212711334229, 'learning_rate': 2.631578947368421e-05, 'epoch': 0.88}
{'loss': 1.444, 'grad_norm': 2.6537492275238037, 'learning_rate': 2.706766917293233e-05, 'epoch': 0.9}
{'loss': 1.4418, 'grad_norm': 4.448942184448242, 'learning_rate': 2.781954887218045e-05, 'epoch': 0.93}
{'loss': 1.4106, 'grad_norm': 5.364701747894287, 'learning_rate': 2.857142857142857e-05, 'epoch': 0.95}
{'loss': 1.3067, 'grad_norm': 23.9955997467041, 'learning_rate': 2.932330827067669e-05, 'epoch': 0.98}
{'eval_loss': 1.2171988487243652, 'eval_accuracy': 0.6208265921459039, 'eval_runtime': 27.8676, 'eval_samples_per_second': 243.975, 'eval_steps_per_second': 7.643, 'epoch': 1.0}
{'loss': 1.3318, 'grad_norm': 8.247232437133789, 'learning_rate': 2.9991645781119465e-05, 'epoch': 1.0}
{'loss': 1.3197, 'grad_norm': 4.268239498138428, 'learning_rate': 2.9908103592314118e-05, 'epoch': 1.03}
{'loss': 1.3609, 'grad_norm': 8.106438636779785, 'learning_rate': 2.9824561403508772e-05, 'epoch': 1.05}
{'loss': 1.2674, 'grad_norm': 4.0687079429626465, 'learning_rate': 2.9741019214703425e-05, 'epoch': 1.08}
{'loss': 1.1788, 'grad_norm': 7.967325687408447, 'learning_rate': 2.965747702589808e-05, 'epoch': 1.1}
{'loss': 1.2682, 'grad_norm': 6.044351577758789, 'learning_rate': 2.9573934837092732e-05, 'epoch': 1.13}
{'loss': 1.2627, 'grad_norm': 8.778580665588379, 'learning_rate': 2.9490392648287386e-05, 'epoch': 1.15}
{'loss': 1.1868, 'grad_norm': 4.018051624298096, 'learning_rate': 2.9406850459482036e-05, 'epoch': 1.18}
{'loss': 1.1715, 'grad_norm': 6.009234428405762, 'learning_rate': 2.932330827067669e-05, 'epoch': 1.2}
{'loss': 1.1471, 'grad_norm': 6.243227005004883, 'learning_rate': 2.9239766081871343e-05, 'epoch': 1.23}
{'loss': 1.121, 'grad_norm': 6.466532230377197, 'learning_rate': 2.9156223893065996e-05, 'epoch': 1.25}
{'loss': 1.1216, 'grad_norm': 16.389780044555664, 'learning_rate': 2.907268170426065e-05, 'epoch': 1.28}
{'loss': 1.1753, 'grad_norm': 14.183526039123535, 'learning_rate': 2.8989139515455303e-05, 'epoch': 1.3}
{'loss': 1.0476, 'grad_norm': 11.895203590393066, 'learning_rate': 2.8905597326649957e-05, 'epoch': 1.33}
{'loss': 1.0994, 'grad_norm': 12.037175178527832, 'learning_rate': 2.882205513784461e-05, 'epoch': 1.35}
{'loss': 1.1211, 'grad_norm': 7.035501480102539, 'learning_rate': 2.8738512949039264e-05, 'epoch': 1.38}
{'loss': 1.0374, 'grad_norm': 6.624691963195801, 'learning_rate': 2.8654970760233917e-05, 'epoch': 1.4}
{'loss': 0.9871, 'grad_norm': 29.647918701171875, 'learning_rate': 2.857142857142857e-05, 'epoch': 1.43}
{'loss': 1.0441, 'grad_norm': 5.806159019470215, 'learning_rate': 2.8487886382623224e-05, 'epoch': 1.45}
{'loss': 1.0124, 'grad_norm': 9.677886009216309, 'learning_rate': 2.8404344193817878e-05, 'epoch': 1.48}
{'loss': 0.9819, 'grad_norm': 32.166622161865234, 'learning_rate': 2.832080200501253e-05, 'epoch': 1.5}
{'loss': 1.0236, 'grad_norm': 9.286630630493164, 'learning_rate': 2.8237259816207185e-05, 'epoch': 1.53}
{'loss': 1.0199, 'grad_norm': 17.63946533203125, 'learning_rate': 2.815371762740184e-05, 'epoch': 1.55}
{'loss': 1.006, 'grad_norm': 54.67157745361328, 'learning_rate': 2.8070175438596492e-05, 'epoch': 1.58}
{'loss': 1.0077, 'grad_norm': 18.511430740356445, 'learning_rate': 2.7986633249791145e-05, 'epoch': 1.6}
{'loss': 0.9834, 'grad_norm': 12.39023208618164, 'learning_rate': 2.7903091060985795e-05, 'epoch': 1.63}
{'loss': 0.9642, 'grad_norm': 14.356637001037598, 'learning_rate': 2.781954887218045e-05, 'epoch': 1.65}
{'loss': 0.9748, 'grad_norm': 11.148791313171387, 'learning_rate': 2.7736006683375102e-05, 'epoch': 1.68}
{'loss': 0.9797, 'grad_norm': 7.136213779449463, 'learning_rate': 2.7652464494569756e-05, 'epoch': 1.7}
{'loss': 0.9452, 'grad_norm': 12.205671310424805, 'learning_rate': 2.756892230576441e-05, 'epoch': 1.73}
{'loss': 0.938, 'grad_norm': 5.833101272583008, 'learning_rate': 2.7485380116959063e-05, 'epoch': 1.75}
{'loss': 0.9212, 'grad_norm': 6.595088958740234, 'learning_rate': 2.7401837928153716e-05, 'epoch': 1.78}
{'loss': 0.9537, 'grad_norm': 6.723524570465088, 'learning_rate': 2.731829573934837e-05, 'epoch': 1.8}
{'loss': 0.9224, 'grad_norm': 18.532451629638672, 'learning_rate': 2.7234753550543023e-05, 'epoch': 1.83}
{'loss': 0.8542, 'grad_norm': 5.355908393859863, 'learning_rate': 2.7151211361737677e-05, 'epoch': 1.85}
{'loss': 0.8263, 'grad_norm': 4.079437732696533, 'learning_rate': 2.706766917293233e-05, 'epoch': 1.88}
{'loss': 0.8462, 'grad_norm': 5.943741798400879, 'learning_rate': 2.6984126984126984e-05, 'epoch': 1.9}
{'loss': 0.8671, 'grad_norm': 4.720602989196777, 'learning_rate': 2.6900584795321637e-05, 'epoch': 1.93}
{'loss': 0.8078, 'grad_norm': 7.28938627243042, 'learning_rate': 2.681704260651629e-05, 'epoch': 1.95}
{'loss': 0.7497, 'grad_norm': 6.738524913787842, 'learning_rate': 2.6733500417710944e-05, 'epoch': 1.98}
{'eval_loss': 0.6217761635780334, 'eval_accuracy': 0.9126342109133696, 'eval_runtime': 28.097, 'eval_samples_per_second': 241.984, 'eval_steps_per_second': 7.581, 'epoch': 2.0}
{'loss': 0.7521, 'grad_norm': 10.50902271270752, 'learning_rate': 2.6649958228905598e-05, 'epoch': 2.0}
{'loss': 0.8101, 'grad_norm': 4.262675762176514, 'learning_rate': 2.656641604010025e-05, 'epoch': 2.03}
{'loss': 0.7696, 'grad_norm': 15.879745483398438, 'learning_rate': 2.64828738512949e-05, 'epoch': 2.05}
{'loss': 0.7518, 'grad_norm': 6.711551189422607, 'learning_rate': 2.6399331662489555e-05, 'epoch': 2.08}
{'loss': 0.7196, 'grad_norm': 24.52581787109375, 'learning_rate': 2.631578947368421e-05, 'epoch': 2.1}
{'loss': 0.7559, 'grad_norm': 21.402677536010742, 'learning_rate': 2.6232247284878862e-05, 'epoch': 2.13}
{'loss': 0.6637, 'grad_norm': 35.80774688720703, 'learning_rate': 2.6148705096073516e-05, 'epoch': 2.15}
{'loss': 0.6637, 'grad_norm': 6.12632417678833, 'learning_rate': 2.606516290726817e-05, 'epoch': 2.18}
{'loss': 0.6651, 'grad_norm': 13.151334762573242, 'learning_rate': 2.5981620718462823e-05, 'epoch': 2.2}
{'loss': 0.6421, 'grad_norm': 5.197573661804199, 'learning_rate': 2.5898078529657476e-05, 'epoch': 2.23}
{'loss': 0.6109, 'grad_norm': 6.187713146209717, 'learning_rate': 2.581453634085213e-05, 'epoch': 2.25}
{'loss': 0.5993, 'grad_norm': 9.095999717712402, 'learning_rate': 2.5730994152046783e-05, 'epoch': 2.28}
{'loss': 0.6046, 'grad_norm': 14.354019165039062, 'learning_rate': 2.5647451963241437e-05, 'epoch': 2.3}
{'loss': 0.5568, 'grad_norm': 4.741723537445068, 'learning_rate': 2.556390977443609e-05, 'epoch': 2.33}
{'loss': 0.5411, 'grad_norm': 10.369399070739746, 'learning_rate': 2.5480367585630744e-05, 'epoch': 2.35}
{'loss': 0.5094, 'grad_norm': 10.91810131072998, 'learning_rate': 2.5396825396825397e-05, 'epoch': 2.38}
{'loss': 0.5595, 'grad_norm': 8.521037101745605, 'learning_rate': 2.531328320802005e-05, 'epoch': 2.4}
{'loss': 0.5421, 'grad_norm': 4.9202775955200195, 'learning_rate': 2.5229741019214704e-05, 'epoch': 2.43}
{'loss': 0.5512, 'grad_norm': 11.397713661193848, 'learning_rate': 2.5146198830409358e-05, 'epoch': 2.45}
{'loss': 0.56, 'grad_norm': 8.038707733154297, 'learning_rate': 2.5062656641604008e-05, 'epoch': 2.48}
{'loss': 0.4863, 'grad_norm': 10.702048301696777, 'learning_rate': 2.497911445279866e-05, 'epoch': 2.5}
{'loss': 0.4967, 'grad_norm': 5.458794116973877, 'learning_rate': 2.4895572263993315e-05, 'epoch': 2.53}
{'loss': 0.5028, 'grad_norm': 4.826409816741943, 'learning_rate': 2.4812030075187968e-05, 'epoch': 2.55}
{'loss': 0.5122, 'grad_norm': 17.02625846862793, 'learning_rate': 2.472848788638262e-05, 'epoch': 2.58}
{'loss': 0.5157, 'grad_norm': 10.512066841125488, 'learning_rate': 2.4644945697577275e-05, 'epoch': 2.6}
{'loss': 0.456, 'grad_norm': 11.934918403625488, 'learning_rate': 2.456140350877193e-05, 'epoch': 2.63}
{'loss': 0.4523, 'grad_norm': 7.118875026702881, 'learning_rate': 2.4477861319966582e-05, 'epoch': 2.65}
{'loss': 0.4853, 'grad_norm': 11.535867691040039, 'learning_rate': 2.4394319131161236e-05, 'epoch': 2.68}
{'loss': 0.4764, 'grad_norm': 12.909754753112793, 'learning_rate': 2.431077694235589e-05, 'epoch': 2.71}
{'loss': 0.4285, 'grad_norm': 6.699863910675049, 'learning_rate': 2.4227234753550543e-05, 'epoch': 2.73}
{'loss': 0.4458, 'grad_norm': 6.415680408477783, 'learning_rate': 2.4143692564745196e-05, 'epoch': 2.76}
{'loss': 0.4333, 'grad_norm': 4.328423976898193, 'learning_rate': 2.406015037593985e-05, 'epoch': 2.78}
{'loss': 0.468, 'grad_norm': 6.400489330291748, 'learning_rate': 2.3976608187134503e-05, 'epoch': 2.81}
{'loss': 0.3812, 'grad_norm': 10.49980640411377, 'learning_rate': 2.3893065998329157e-05, 'epoch': 2.83}
{'loss': 0.4521, 'grad_norm': 9.107507705688477, 'learning_rate': 2.380952380952381e-05, 'epoch': 2.86}
{'loss': 0.426, 'grad_norm': 8.427456855773926, 'learning_rate': 2.3725981620718464e-05, 'epoch': 2.88}
{'loss': 0.4097, 'grad_norm': 4.324967384338379, 'learning_rate': 2.3642439431913114e-05, 'epoch': 2.91}
{'loss': 0.3497, 'grad_norm': 7.4743781089782715, 'learning_rate': 2.3558897243107767e-05, 'epoch': 2.93}
{'loss': 0.4074, 'grad_norm': 5.867978572845459, 'learning_rate': 2.347535505430242e-05, 'epoch': 2.96}
{'loss': 0.4221, 'grad_norm': 15.518388748168945, 'learning_rate': 2.3391812865497074e-05, 'epoch': 2.98}
{'eval_loss': 0.215253546833992, 'eval_accuracy': 0.9741138402706281, 'eval_runtime': 28.0641, 'eval_samples_per_second': 242.267, 'eval_steps_per_second': 7.59, 'epoch': 3.0}
{'loss': 0.3508, 'grad_norm': 6.627721309661865, 'learning_rate': 2.3308270676691728e-05, 'epoch': 3.01}
{'loss': 0.3727, 'grad_norm': 50.13751983642578, 'learning_rate': 2.322472848788638e-05, 'epoch': 3.03}
{'loss': 0.39, 'grad_norm': 9.700447082519531, 'learning_rate': 2.3141186299081035e-05, 'epoch': 3.06}
{'loss': 0.3451, 'grad_norm': 13.40552806854248, 'learning_rate': 2.3057644110275688e-05, 'epoch': 3.08}
{'loss': 0.3561, 'grad_norm': 12.970911979675293, 'learning_rate': 2.2974101921470342e-05, 'epoch': 3.11}
{'loss': 0.3228, 'grad_norm': 7.73871374130249, 'learning_rate': 2.2890559732664995e-05, 'epoch': 3.13}
{'loss': 0.3501, 'grad_norm': 22.470598220825195, 'learning_rate': 2.280701754385965e-05, 'epoch': 3.16}
{'loss': 0.338, 'grad_norm': 4.909379482269287, 'learning_rate': 2.2723475355054302e-05, 'epoch': 3.18}
{'loss': 0.2972, 'grad_norm': 12.108818054199219, 'learning_rate': 2.2639933166248956e-05, 'epoch': 3.21}
{'loss': 0.3353, 'grad_norm': 4.764550685882568, 'learning_rate': 2.255639097744361e-05, 'epoch': 3.23}
{'loss': 0.314, 'grad_norm': 35.886451721191406, 'learning_rate': 2.2472848788638263e-05, 'epoch': 3.26}
{'loss': 0.2681, 'grad_norm': 7.195383548736572, 'learning_rate': 2.2389306599832916e-05, 'epoch': 3.28}
{'loss': 0.301, 'grad_norm': 10.322922706604004, 'learning_rate': 2.230576441102757e-05, 'epoch': 3.31}
{'loss': 0.292, 'grad_norm': 7.155256271362305, 'learning_rate': 2.222222222222222e-05, 'epoch': 3.33}
{'loss': 0.2878, 'grad_norm': 6.9950642585754395, 'learning_rate': 2.2138680033416873e-05, 'epoch': 3.36}
{'loss': 0.2841, 'grad_norm': 14.657593727111816, 'learning_rate': 2.2055137844611527e-05, 'epoch': 3.38}
{'loss': 0.2855, 'grad_norm': 46.04249954223633, 'learning_rate': 2.197159565580618e-05, 'epoch': 3.41}
{'loss': 0.3189, 'grad_norm': 5.4169464111328125, 'learning_rate': 2.1888053467000834e-05, 'epoch': 3.43}
{'loss': 0.2915, 'grad_norm': 7.981625556945801, 'learning_rate': 2.1804511278195487e-05, 'epoch': 3.46}
{'loss': 0.2941, 'grad_norm': 7.316673278808594, 'learning_rate': 2.172096908939014e-05, 'epoch': 3.48}
{'loss': 0.3061, 'grad_norm': 22.258466720581055, 'learning_rate': 2.1637426900584794e-05, 'epoch': 3.51}
{'loss': 0.2783, 'grad_norm': 5.324995517730713, 'learning_rate': 2.1553884711779448e-05, 'epoch': 3.53}
{'loss': 0.3188, 'grad_norm': 13.883384704589844, 'learning_rate': 2.14703425229741e-05, 'epoch': 3.56}
{'loss': 0.2465, 'grad_norm': 5.2524237632751465, 'learning_rate': 2.1386800334168755e-05, 'epoch': 3.58}
{'loss': 0.3202, 'grad_norm': 15.477888107299805, 'learning_rate': 2.130325814536341e-05, 'epoch': 3.61}
{'loss': 0.249, 'grad_norm': 14.159198760986328, 'learning_rate': 2.1219715956558062e-05, 'epoch': 3.63}
{'loss': 0.3029, 'grad_norm': 14.150718688964844, 'learning_rate': 2.1136173767752715e-05, 'epoch': 3.66}
{'loss': 0.2706, 'grad_norm': 10.32032299041748, 'learning_rate': 2.105263157894737e-05, 'epoch': 3.68}
{'loss': 0.3113, 'grad_norm': 3.8503079414367676, 'learning_rate': 2.0969089390142022e-05, 'epoch': 3.71}
{'loss': 0.3024, 'grad_norm': 7.238915920257568, 'learning_rate': 2.0885547201336676e-05, 'epoch': 3.73}
{'loss': 0.2613, 'grad_norm': 8.796319007873535, 'learning_rate': 2.0802005012531326e-05, 'epoch': 3.76}
{'loss': 0.2784, 'grad_norm': 10.822440147399902, 'learning_rate': 2.071846282372598e-05, 'epoch': 3.78}
{'loss': 0.2596, 'grad_norm': 22.921066284179688, 'learning_rate': 2.0634920634920633e-05, 'epoch': 3.81}
{'loss': 0.2231, 'grad_norm': 5.323471546173096, 'learning_rate': 2.0551378446115287e-05, 'epoch': 3.83}
{'loss': 0.2783, 'grad_norm': 12.948866844177246, 'learning_rate': 2.046783625730994e-05, 'epoch': 3.86}
{'loss': 0.2929, 'grad_norm': 12.70848274230957, 'learning_rate': 2.0384294068504594e-05, 'epoch': 3.88}
{'loss': 0.2592, 'grad_norm': 4.705185413360596, 'learning_rate': 2.0300751879699247e-05, 'epoch': 3.91}
{'loss': 0.2565, 'grad_norm': 9.191632270812988, 'learning_rate': 2.02172096908939e-05, 'epoch': 3.93}
{'loss': 0.2462, 'grad_norm': 9.111980438232422, 'learning_rate': 2.0133667502088554e-05, 'epoch': 3.96}
{'loss': 0.2623, 'grad_norm': 3.964816093444824, 'learning_rate': 2.0050125313283208e-05, 'epoch': 3.98}
{'eval_loss': 0.12593592703342438, 'eval_accuracy': 0.9786733343138697, 'eval_runtime': 27.7517, 'eval_samples_per_second': 244.994, 'eval_steps_per_second': 7.675, 'epoch': 4.0}
{'loss': 0.2308, 'grad_norm': 5.410640716552734, 'learning_rate': 1.996658312447786e-05, 'epoch': 4.01}
{'loss': 0.2606, 'grad_norm': 5.7246012687683105, 'learning_rate': 1.9883040935672515e-05, 'epoch': 4.03}
{'loss': 0.214, 'grad_norm': 8.236963272094727, 'learning_rate': 1.9799498746867168e-05, 'epoch': 4.06}
{'loss': 0.2591, 'grad_norm': 3.9621870517730713, 'learning_rate': 1.971595655806182e-05, 'epoch': 4.08}
{'loss': 0.2228, 'grad_norm': 5.975111961364746, 'learning_rate': 1.9632414369256475e-05, 'epoch': 4.11}
{'loss': 0.2099, 'grad_norm': 6.177577972412109, 'learning_rate': 1.954887218045113e-05, 'epoch': 4.13}
{'loss': 0.2526, 'grad_norm': 14.805758476257324, 'learning_rate': 1.9465329991645782e-05, 'epoch': 4.16}
{'loss': 0.2404, 'grad_norm': 4.188657283782959, 'learning_rate': 1.9381787802840436e-05, 'epoch': 4.18}
{'loss': 0.2062, 'grad_norm': 7.93744421005249, 'learning_rate': 1.9298245614035086e-05, 'epoch': 4.21}
{'loss': 0.2102, 'grad_norm': 7.097870349884033, 'learning_rate': 1.921470342522974e-05, 'epoch': 4.23}
{'loss': 0.2593, 'grad_norm': 5.4622626304626465, 'learning_rate': 1.9131161236424393e-05, 'epoch': 4.26}
{'loss': 0.2425, 'grad_norm': 4.448942184448242, 'learning_rate': 1.9047619047619046e-05, 'epoch': 4.28}
{'loss': 0.2359, 'grad_norm': 5.490296840667725, 'learning_rate': 1.89640768588137e-05, 'epoch': 4.31}
{'loss': 0.2294, 'grad_norm': 6.866471290588379, 'learning_rate': 1.8880534670008353e-05, 'epoch': 4.33}
{'loss': 0.2335, 'grad_norm': 6.195837497711182, 'learning_rate': 1.8796992481203007e-05, 'epoch': 4.36}
{'loss': 0.2614, 'grad_norm': 6.977581977844238, 'learning_rate': 1.871345029239766e-05, 'epoch': 4.38}
{'loss': 0.2147, 'grad_norm': 9.559826850891113, 'learning_rate': 1.8629908103592314e-05, 'epoch': 4.41}
{'loss': 0.18, 'grad_norm': 4.037659168243408, 'learning_rate': 1.8546365914786967e-05, 'epoch': 4.43}
{'loss': 0.2435, 'grad_norm': 11.133934020996094, 'learning_rate': 1.846282372598162e-05, 'epoch': 4.46}
{'loss': 0.2305, 'grad_norm': 29.02533721923828, 'learning_rate': 1.8379281537176274e-05, 'epoch': 4.48}
{'loss': 0.2224, 'grad_norm': 13.827043533325195, 'learning_rate': 1.8295739348370928e-05, 'epoch': 4.51}
{'loss': 0.2843, 'grad_norm': 5.777353286743164, 'learning_rate': 1.821219715956558e-05, 'epoch': 4.53}
{'loss': 0.2187, 'grad_norm': 12.559530258178711, 'learning_rate': 1.8128654970760235e-05, 'epoch': 4.56}
{'loss': 0.259, 'grad_norm': 11.86184024810791, 'learning_rate': 1.8045112781954888e-05, 'epoch': 4.58}
{'loss': 0.1787, 'grad_norm': 42.00202178955078, 'learning_rate': 1.796157059314954e-05, 'epoch': 4.61}
{'loss': 0.1882, 'grad_norm': 11.309063911437988, 'learning_rate': 1.7878028404344192e-05, 'epoch': 4.63}
{'loss': 0.1994, 'grad_norm': 8.91698169708252, 'learning_rate': 1.7794486215538845e-05, 'epoch': 4.66}
{'loss': 0.2075, 'grad_norm': 5.6022233963012695, 'learning_rate': 1.77109440267335e-05, 'epoch': 4.68}
{'loss': 0.1915, 'grad_norm': 10.359853744506836, 'learning_rate': 1.7627401837928152e-05, 'epoch': 4.71}
{'loss': 0.2009, 'grad_norm': 10.575214385986328, 'learning_rate': 1.7543859649122806e-05, 'epoch': 4.73}
{'loss': 0.1952, 'grad_norm': 6.624807357788086, 'learning_rate': 1.746031746031746e-05, 'epoch': 4.76}
{'loss': 0.2102, 'grad_norm': 17.153003692626953, 'learning_rate': 1.7376775271512113e-05, 'epoch': 4.78}
{'loss': 0.1957, 'grad_norm': 6.653409957885742, 'learning_rate': 1.7293233082706766e-05, 'epoch': 4.81}
{'loss': 0.1858, 'grad_norm': 6.631964206695557, 'learning_rate': 1.720969089390142e-05, 'epoch': 4.83}
{'loss': 0.1837, 'grad_norm': 7.057446002960205, 'learning_rate': 1.7126148705096073e-05, 'epoch': 4.86}
{'loss': 0.2139, 'grad_norm': 9.704708099365234, 'learning_rate': 1.7042606516290727e-05, 'epoch': 4.88}
{'loss': 0.1889, 'grad_norm': 4.732715606689453, 'learning_rate': 1.695906432748538e-05, 'epoch': 4.91}
{'loss': 0.2426, 'grad_norm': 7.031479835510254, 'learning_rate': 1.6875522138680034e-05, 'epoch': 4.93}
{'loss': 0.2363, 'grad_norm': 4.444036483764648, 'learning_rate': 1.6791979949874687e-05, 'epoch': 4.96}
{'loss': 0.2174, 'grad_norm': 4.470202922821045, 'learning_rate': 1.670843776106934e-05, 'epoch': 4.98}
{'eval_loss': 0.10668482631444931, 'eval_accuracy': 0.9794087365789087, 'eval_runtime': 27.8109, 'eval_samples_per_second': 244.472, 'eval_steps_per_second': 7.659, 'epoch': 5.0}
{'loss': 0.1864, 'grad_norm': 7.008035182952881, 'learning_rate': 1.6624895572263994e-05, 'epoch': 5.01}
{'loss': 0.2369, 'grad_norm': 9.074469566345215, 'learning_rate': 1.6541353383458648e-05, 'epoch': 5.03}
{'loss': 0.1838, 'grad_norm': 6.960032939910889, 'learning_rate': 1.6457811194653298e-05, 'epoch': 5.06}
{'loss': 0.2143, 'grad_norm': 9.120850563049316, 'learning_rate': 1.637426900584795e-05, 'epoch': 5.08}
{'loss': 0.1974, 'grad_norm': 17.08858299255371, 'learning_rate': 1.6290726817042605e-05, 'epoch': 5.11}
{'loss': 0.156, 'grad_norm': 6.9683756828308105, 'learning_rate': 1.620718462823726e-05, 'epoch': 5.13}
{'loss': 0.1881, 'grad_norm': 6.124068737030029, 'learning_rate': 1.6123642439431912e-05, 'epoch': 5.16}
{'loss': 0.2699, 'grad_norm': 38.71773147583008, 'learning_rate': 1.6040100250626565e-05, 'epoch': 5.18}
{'loss': 0.1604, 'grad_norm': 6.267723560333252, 'learning_rate': 1.595655806182122e-05, 'epoch': 5.21}
{'loss': 0.2295, 'grad_norm': 11.4260835647583, 'learning_rate': 1.5873015873015872e-05, 'epoch': 5.23}
{'loss': 0.1918, 'grad_norm': 5.027204990386963, 'learning_rate': 1.5789473684210526e-05, 'epoch': 5.26}
{'loss': 0.1805, 'grad_norm': 4.081631660461426, 'learning_rate': 1.570593149540518e-05, 'epoch': 5.28}
{'loss': 0.1876, 'grad_norm': 2.020601511001587, 'learning_rate': 1.5622389306599833e-05, 'epoch': 5.31}
{'loss': 0.1735, 'grad_norm': 8.568584442138672, 'learning_rate': 1.5538847117794486e-05, 'epoch': 5.34}
{'loss': 0.2216, 'grad_norm': 4.930680751800537, 'learning_rate': 1.545530492898914e-05, 'epoch': 5.36}
{'loss': 0.2216, 'grad_norm': 6.274499893188477, 'learning_rate': 1.5371762740183793e-05, 'epoch': 5.39}
{'loss': 0.1637, 'grad_norm': 5.239953517913818, 'learning_rate': 1.5288220551378447e-05, 'epoch': 5.41}
{'loss': 0.1963, 'grad_norm': 6.393955230712891, 'learning_rate': 1.5204678362573099e-05, 'epoch': 5.44}
{'loss': 0.2092, 'grad_norm': 12.013360977172852, 'learning_rate': 1.5121136173767752e-05, 'epoch': 5.46}
{'loss': 0.1986, 'grad_norm': 4.146913051605225, 'learning_rate': 1.5037593984962406e-05, 'epoch': 5.49}
{'loss': 0.1969, 'grad_norm': 10.29556941986084, 'learning_rate': 1.4954051796157059e-05, 'epoch': 5.51}
{'loss': 0.1603, 'grad_norm': 18.80317497253418, 'learning_rate': 1.4870509607351713e-05, 'epoch': 5.54}
{'loss': 0.2294, 'grad_norm': 9.191271781921387, 'learning_rate': 1.4786967418546366e-05, 'epoch': 5.56}
{'loss': 0.1821, 'grad_norm': 3.955827236175537, 'learning_rate': 1.4703425229741018e-05, 'epoch': 5.59}
{'loss': 0.2141, 'grad_norm': 15.517136573791504, 'learning_rate': 1.4619883040935671e-05, 'epoch': 5.61}
{'loss': 0.1728, 'grad_norm': 7.475724697113037, 'learning_rate': 1.4536340852130325e-05, 'epoch': 5.64}
{'loss': 0.1578, 'grad_norm': 7.843225002288818, 'learning_rate': 1.4452798663324978e-05, 'epoch': 5.66}
{'loss': 0.1834, 'grad_norm': 6.4552693367004395, 'learning_rate': 1.4369256474519632e-05, 'epoch': 5.69}
{'loss': 0.1746, 'grad_norm': 4.760468006134033, 'learning_rate': 1.4285714285714285e-05, 'epoch': 5.71}
{'loss': 0.2047, 'grad_norm': 4.731630325317383, 'learning_rate': 1.4202172096908939e-05, 'epoch': 5.74}
{'loss': 0.198, 'grad_norm': 2.367713451385498, 'learning_rate': 1.4118629908103592e-05, 'epoch': 5.76}
{'loss': 0.1634, 'grad_norm': 63.4054069519043, 'learning_rate': 1.4035087719298246e-05, 'epoch': 5.79}
{'loss': 0.1502, 'grad_norm': 5.804986000061035, 'learning_rate': 1.3951545530492898e-05, 'epoch': 5.81}
{'loss': 0.167, 'grad_norm': 5.287708282470703, 'learning_rate': 1.3868003341687551e-05, 'epoch': 5.84}
{'loss': 0.207, 'grad_norm': 10.64536190032959, 'learning_rate': 1.3784461152882205e-05, 'epoch': 5.86}
{'loss': 0.1794, 'grad_norm': 7.4702229499816895, 'learning_rate': 1.3700918964076858e-05, 'epoch': 5.89}
{'loss': 0.1666, 'grad_norm': 9.817946434020996, 'learning_rate': 1.3617376775271512e-05, 'epoch': 5.91}
{'loss': 0.168, 'grad_norm': 6.233557224273682, 'learning_rate': 1.3533834586466165e-05, 'epoch': 5.94}
{'loss': 0.1961, 'grad_norm': 19.743722915649414, 'learning_rate': 1.3450292397660819e-05, 'epoch': 5.96}
{'loss': 0.2175, 'grad_norm': 11.29397964477539, 'learning_rate': 1.3366750208855472e-05, 'epoch': 5.99}
{'eval_loss': 0.0899258479475975, 'eval_accuracy': 0.9816149433740256, 'eval_runtime': 28.1831, 'eval_samples_per_second': 241.244, 'eval_steps_per_second': 7.558, 'epoch': 6.0}
{'loss': 0.1478, 'grad_norm': 5.930534362792969, 'learning_rate': 1.3283208020050126e-05, 'epoch': 6.01}
{'loss': 0.1427, 'grad_norm': 6.262040138244629, 'learning_rate': 1.3199665831244778e-05, 'epoch': 6.04}
{'loss': 0.1543, 'grad_norm': 2.7355828285217285, 'learning_rate': 1.3116123642439431e-05, 'epoch': 6.06}
{'loss': 0.1851, 'grad_norm': 11.643403053283691, 'learning_rate': 1.3032581453634085e-05, 'epoch': 6.09}
{'loss': 0.1742, 'grad_norm': 19.47243881225586, 'learning_rate': 1.2949039264828738e-05, 'epoch': 6.11}
{'loss': 0.187, 'grad_norm': 9.586103439331055, 'learning_rate': 1.2865497076023392e-05, 'epoch': 6.14}
{'loss': 0.1627, 'grad_norm': 10.24963665008545, 'learning_rate': 1.2781954887218045e-05, 'epoch': 6.16}
{'loss': 0.159, 'grad_norm': 7.507512092590332, 'learning_rate': 1.2698412698412699e-05, 'epoch': 6.19}
{'loss': 0.1717, 'grad_norm': 8.502533912658691, 'learning_rate': 1.2614870509607352e-05, 'epoch': 6.21}
{'loss': 0.2045, 'grad_norm': 6.647087097167969, 'learning_rate': 1.2531328320802004e-05, 'epoch': 6.24}
{'loss': 0.1879, 'grad_norm': 8.851253509521484, 'learning_rate': 1.2447786131996657e-05, 'epoch': 6.26}
{'loss': 0.2007, 'grad_norm': 4.650445461273193, 'learning_rate': 1.236424394319131e-05, 'epoch': 6.29}
{'loss': 0.1511, 'grad_norm': 3.3397068977355957, 'learning_rate': 1.2280701754385964e-05, 'epoch': 6.31}
{'loss': 0.1622, 'grad_norm': 10.059924125671387, 'learning_rate': 1.2197159565580618e-05, 'epoch': 6.34}
{'loss': 0.1559, 'grad_norm': 4.451300144195557, 'learning_rate': 1.2113617376775271e-05, 'epoch': 6.36}
{'loss': 0.1487, 'grad_norm': 4.742477893829346, 'learning_rate': 1.2030075187969925e-05, 'epoch': 6.39}
{'loss': 0.2047, 'grad_norm': 10.4263334274292, 'learning_rate': 1.1946532999164578e-05, 'epoch': 6.41}
{'loss': 0.1605, 'grad_norm': 4.790078639984131, 'learning_rate': 1.1862990810359232e-05, 'epoch': 6.44}
{'loss': 0.1976, 'grad_norm': 19.83657455444336, 'learning_rate': 1.1779448621553884e-05, 'epoch': 6.46}
{'loss': 0.1722, 'grad_norm': 2.925044298171997, 'learning_rate': 1.1695906432748537e-05, 'epoch': 6.49}
{'loss': 0.1949, 'grad_norm': 5.578660488128662, 'learning_rate': 1.161236424394319e-05, 'epoch': 6.51}
{'loss': 0.1762, 'grad_norm': 14.68494701385498, 'learning_rate': 1.1528822055137844e-05, 'epoch': 6.54}
{'loss': 0.1559, 'grad_norm': 7.540688991546631, 'learning_rate': 1.1445279866332498e-05, 'epoch': 6.56}
{'loss': 0.1886, 'grad_norm': 5.188674449920654, 'learning_rate': 1.1361737677527151e-05, 'epoch': 6.59}
{'loss': 0.1632, 'grad_norm': 5.7409820556640625, 'learning_rate': 1.1278195488721805e-05, 'epoch': 6.61}
{'loss': 0.154, 'grad_norm': 6.383751392364502, 'learning_rate': 1.1194653299916458e-05, 'epoch': 6.64}
{'loss': 0.1684, 'grad_norm': 6.699789047241211, 'learning_rate': 1.111111111111111e-05, 'epoch': 6.66}
{'loss': 0.2009, 'grad_norm': 9.15067195892334, 'learning_rate': 1.1027568922305763e-05, 'epoch': 6.69}
{'loss': 0.1608, 'grad_norm': 11.577309608459473, 'learning_rate': 1.0944026733500417e-05, 'epoch': 6.71}
{'loss': 0.1741, 'grad_norm': 5.671051502227783, 'learning_rate': 1.086048454469507e-05, 'epoch': 6.74}
{'loss': 0.1734, 'grad_norm': 5.868266582489014, 'learning_rate': 1.0776942355889724e-05, 'epoch': 6.76}
{'loss': 0.1738, 'grad_norm': 7.380395412445068, 'learning_rate': 1.0693400167084377e-05, 'epoch': 6.79}
{'loss': 0.1295, 'grad_norm': 10.68338394165039, 'learning_rate': 1.0609857978279031e-05, 'epoch': 6.81}
{'loss': 0.1527, 'grad_norm': 4.508953094482422, 'learning_rate': 1.0526315789473684e-05, 'epoch': 6.84}
{'loss': 0.1842, 'grad_norm': 7.8929924964904785, 'learning_rate': 1.0442773600668338e-05, 'epoch': 6.86}
{'loss': 0.1803, 'grad_norm': 5.5200114250183105, 'learning_rate': 1.035923141186299e-05, 'epoch': 6.89}
{'loss': 0.1662, 'grad_norm': 8.556962013244629, 'learning_rate': 1.0275689223057643e-05, 'epoch': 6.91}
{'loss': 0.1768, 'grad_norm': 4.2880401611328125, 'learning_rate': 1.0192147034252297e-05, 'epoch': 6.94}
{'loss': 0.1641, 'grad_norm': 9.29998779296875, 'learning_rate': 1.010860484544695e-05, 'epoch': 6.96}
{'loss': 0.1709, 'grad_norm': 5.35491418838501, 'learning_rate': 1.0025062656641604e-05, 'epoch': 6.99}
{'eval_loss': 0.07824306935071945, 'eval_accuracy': 0.9826445065450802, 'eval_runtime': 28.0126, 'eval_samples_per_second': 242.712, 'eval_steps_per_second': 7.604, 'epoch': 7.0}
{'loss': 0.1682, 'grad_norm': 3.039179801940918, 'learning_rate': 9.941520467836257e-06, 'epoch': 7.01}
{'loss': 0.1494, 'grad_norm': 21.345251083374023, 'learning_rate': 9.85797827903091e-06, 'epoch': 7.04}
{'loss': 0.1542, 'grad_norm': 5.100749969482422, 'learning_rate': 9.774436090225564e-06, 'epoch': 7.06}
{'loss': 0.1623, 'grad_norm': 2.389859676361084, 'learning_rate': 9.690893901420218e-06, 'epoch': 7.09}
{'loss': 0.1696, 'grad_norm': 6.218756675720215, 'learning_rate': 9.60735171261487e-06, 'epoch': 7.11}
{'loss': 0.1307, 'grad_norm': 10.864054679870605, 'learning_rate': 9.523809523809523e-06, 'epoch': 7.14}
{'loss': 0.1614, 'grad_norm': 5.650452613830566, 'learning_rate': 9.440267335004177e-06, 'epoch': 7.16}
{'loss': 0.1675, 'grad_norm': 6.56540060043335, 'learning_rate': 9.35672514619883e-06, 'epoch': 7.19}
{'loss': 0.1594, 'grad_norm': 3.7413723468780518, 'learning_rate': 8.35421888053467e-06, 'epoch': 7.49}
{'loss': 0.1547, 'grad_norm': 11.02135944366455, 'learning_rate': 8.270676691729324e-06, 'epoch': 7.51}
{'loss': 0.1554, 'grad_norm': 9.668456077575684, 'learning_rate': 8.187134502923976e-06, 'epoch': 7.54}
{'loss': 0.1471, 'grad_norm': 4.255740642547607, 'learning_rate': 8.10359231411863e-06, 'epoch': 7.56}
{'loss': 0.1371, 'grad_norm': 10.252484321594238, 'learning_rate': 8.020050125313283e-06, 'epoch': 7.59}
{'loss': 0.1762, 'grad_norm': 9.173072814941406, 'learning_rate': 7.936507936507936e-06, 'epoch': 7.61}
{'loss': 0.1957, 'grad_norm': 13.26845645904541, 'learning_rate': 7.85296574770259e-06, 'epoch': 7.64}
{'loss': 0.1609, 'grad_norm': 8.434731483459473, 'learning_rate': 7.769423558897243e-06, 'epoch': 7.66}
{'loss': 0.1043, 'grad_norm': 6.934642791748047, 'learning_rate': 7.685881370091897e-06, 'epoch': 7.69}
 77%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                      | 3079/3990 [1:27:34<25:06,  1.65s/it]  77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                    | 3080/3990 [1:27:36<24:54,  1.64s/it]
{'loss': 0.1707, 'grad_norm': 20.553653717041016, 'learning_rate': 7.602339181286549e-06, 'epoch': 7.71}
{'loss': 0.1747, 'grad_norm': 5.260369777679443, 'learning_rate': 7.518796992481203e-06, 'epoch': 7.74}
{'loss': 0.1407, 'grad_norm': 6.877122402191162, 'learning_rate': 7.435254803675856e-06, 'epoch': 7.76}
{'loss': 0.1521, 'grad_norm': 3.3921878337860107, 'learning_rate': 7.351712614870509e-06, 'epoch': 7.79}
{'loss': 0.1571, 'grad_norm': 2.8220109939575195, 'learning_rate': 7.2681704260651625e-06, 'epoch': 7.81}
{'loss': 0.1657, 'grad_norm': 3.226956844329834, 'learning_rate': 7.184628237259816e-06, 'epoch': 7.84}
{'loss': 0.1504, 'grad_norm': 19.696043014526367, 'learning_rate': 7.1010860484544695e-06, 'epoch': 7.86}
{'loss': 0.1337, 'grad_norm': 5.227226734161377, 'learning_rate': 7.017543859649123e-06, 'epoch': 7.89}
{'loss': 0.1312, 'grad_norm': 5.866220474243164, 'learning_rate': 6.934001670843776e-06, 'epoch': 7.91}
{'loss': 0.1291, 'grad_norm': 2.414639711380005, 'learning_rate': 6.850459482038429e-06, 'epoch': 7.94}
{'loss': 0.1785, 'grad_norm': 6.379312515258789, 'learning_rate': 6.766917293233083e-06, 'epoch': 7.96}
{'loss': 0.1663, 'grad_norm': 5.101008415222168, 'learning_rate': 6.683375104427736e-06, 'epoch': 7.99}
{'eval_loss': 0.06973809003829956, 'eval_accuracy': 0.9836740697161347, 'eval_runtime': 27.9757, 'eval_samples_per_second': 243.033, 'eval_steps_per_second': 7.614, 'epoch': 8.0}
{'loss': 0.1531, 'grad_norm': 13.97124195098877, 'learning_rate': 6.599832915622389e-06, 'epoch': 8.02}
{'loss': 0.153, 'grad_norm': 15.842631340026855, 'learning_rate': 6.516290726817042e-06, 'epoch': 8.04}
{'loss': 0.1531, 'grad_norm': 20.626882553100586, 'learning_rate': 6.432748538011696e-06, 'epoch': 8.07}
{'loss': 0.1309, 'grad_norm': 6.734814167022705, 'learning_rate': 6.349206349206349e-06, 'epoch': 8.09}
{'loss': 0.1485, 'grad_norm': 13.649189949035645, 'learning_rate': 6.265664160401002e-06, 'epoch': 8.12}
{'loss': 0.1309, 'grad_norm': 6.077277660369873, 'learning_rate': 6.182121971595655e-06, 'epoch': 8.14}
{'loss': 0.1167, 'grad_norm': 4.172124862670898, 'learning_rate': 6.098579782790309e-06, 'epoch': 8.17}
{'loss': 0.1549, 'grad_norm': 11.708206176757812, 'learning_rate': 6.015037593984962e-06, 'epoch': 8.19}
{'loss': 0.1489, 'grad_norm': 9.751978874206543, 'learning_rate': 5.931495405179616e-06, 'epoch': 8.22}
{'loss': 0.1521, 'grad_norm': 5.691232681274414, 'learning_rate': 5.8479532163742686e-06, 'epoch': 8.24}
{'loss': 0.1644, 'grad_norm': 8.01517391204834, 'learning_rate': 5.764411027568922e-06, 'epoch': 8.27}
{'loss': 0.1661, 'grad_norm': 4.623439788818359, 'learning_rate': 5.6808688387635756e-06, 'epoch': 8.29}
{'loss': 0.1578, 'grad_norm': 10.336190223693848, 'learning_rate': 5.597326649958229e-06, 'epoch': 8.32}
{'loss': 0.1562, 'grad_norm': 5.093812942504883, 'learning_rate': 5.513784461152882e-06, 'epoch': 8.34}
{'loss': 0.1361, 'grad_norm': 11.706295013427734, 'learning_rate': 5.430242272347535e-06, 'epoch': 8.37}
{'loss': 0.1259, 'grad_norm': 6.628255844116211, 'learning_rate': 5.346700083542189e-06, 'epoch': 8.39}
{'loss': 0.1392, 'grad_norm': 5.608328342437744, 'learning_rate': 5.263157894736842e-06, 'epoch': 8.42}
{'loss': 0.1643, 'grad_norm': 9.598685264587402, 'learning_rate': 5.179615705931495e-06, 'epoch': 8.44}
{'loss': 0.1433, 'grad_norm': 5.223998546600342, 'learning_rate': 5.096073517126148e-06, 'epoch': 8.47}
{'loss': 0.1518, 'grad_norm': 3.0823729038238525, 'learning_rate': 5.012531328320802e-06, 'epoch': 8.49}
{'loss': 0.1259, 'grad_norm': 3.7573318481445312, 'learning_rate': 4.928989139515455e-06, 'epoch': 8.52}
{'loss': 0.148, 'grad_norm': 6.297669887542725, 'learning_rate': 4.845446950710109e-06, 'epoch': 8.54}
{'loss': 0.1591, 'grad_norm': 5.845996856689453, 'learning_rate': 4.7619047619047615e-06, 'epoch': 8.57}
{'loss': 0.1472, 'grad_norm': 5.604744911193848, 'learning_rate': 4.678362573099415e-06, 'epoch': 8.59}
{'loss': 0.1324, 'grad_norm': 5.2654032707214355, 'learning_rate': 4.5948203842940685e-06, 'epoch': 8.62}
{'loss': 0.1462, 'grad_norm': 9.455986022949219, 'learning_rate': 4.511278195488722e-06, 'epoch': 8.64}
{'loss': 0.134, 'grad_norm': 4.391998767852783, 'learning_rate': 4.427736006683375e-06, 'epoch': 8.67}
{'loss': 0.1452, 'grad_norm': 2.7452847957611084, 'learning_rate': 4.344193817878028e-06, 'epoch': 8.69}
{'loss': 0.1274, 'grad_norm': 5.382234573364258, 'learning_rate': 4.260651629072682e-06, 'epoch': 8.72}
{'loss': 0.1306, 'grad_norm': 12.038947105407715, 'learning_rate': 4.177109440267335e-06, 'epoch': 8.74}
{'loss': 0.1979, 'grad_norm': 9.72067642211914, 'learning_rate': 4.093567251461988e-06, 'epoch': 8.77}
{'loss': 0.1683, 'grad_norm': 9.362449645996094, 'learning_rate': 4.010025062656641e-06, 'epoch': 8.79}
{'loss': 0.1681, 'grad_norm': 9.070378303527832, 'learning_rate': 3.926482873851295e-06, 'epoch': 8.82}
{'loss': 0.1802, 'grad_norm': 10.064006805419922, 'learning_rate': 3.842940685045948e-06, 'epoch': 8.84}
{'loss': 0.1465, 'grad_norm': 10.951995849609375, 'learning_rate': 3.7593984962406014e-06, 'epoch': 8.87}
{'loss': 0.1047, 'grad_norm': 4.666315078735352, 'learning_rate': 3.6758563074352545e-06, 'epoch': 8.89}
{'loss': 0.1699, 'grad_norm': 11.0365571975708, 'learning_rate': 3.592314118629908e-06, 'epoch': 8.92}
{'loss': 0.1179, 'grad_norm': 7.187963008880615, 'learning_rate': 3.5087719298245615e-06, 'epoch': 8.94}
{'loss': 0.1431, 'grad_norm': 10.553169250488281, 'learning_rate': 3.4252297410192146e-06, 'epoch': 8.97}
{'loss': 0.1123, 'grad_norm': 0.9125076532363892, 'learning_rate': 3.341687552213868e-06, 'epoch': 8.99}
{'eval_loss': 0.07648200541734695, 'eval_accuracy': 0.9836740697161347, 'eval_runtime': 27.5382, 'eval_samples_per_second': 246.893, 'eval_steps_per_second': 7.735, 'epoch': 9.0}
{'loss': 0.1684, 'grad_norm': 4.440717697143555, 'learning_rate': 3.258145363408521e-06, 'epoch': 9.02}
{'loss': 0.138, 'grad_norm': 6.505924224853516, 'learning_rate': 3.1746031746031746e-06, 'epoch': 9.04}
{'loss': 0.1507, 'grad_norm': 1.4331977367401123, 'learning_rate': 3.0910609857978277e-06, 'epoch': 9.07}
{'loss': 0.1623, 'grad_norm': 3.832676410675049, 'learning_rate': 3.007518796992481e-06, 'epoch': 9.09}
{'loss': 0.1276, 'grad_norm': 5.004692554473877, 'learning_rate': 2.9239766081871343e-06, 'epoch': 9.12}
{'loss': 0.1499, 'grad_norm': 2.8013219833374023, 'learning_rate': 2.8404344193817878e-06, 'epoch': 9.14}
{'loss': 0.112, 'grad_norm': 9.572582244873047, 'learning_rate': 2.756892230576441e-06, 'epoch': 9.17}
{'loss': 0.1221, 'grad_norm': 2.483337640762329, 'learning_rate': 2.6733500417710944e-06, 'epoch': 9.19}
{'loss': 0.1232, 'grad_norm': 8.162860870361328, 'learning_rate': 2.5898078529657474e-06, 'epoch': 9.22}
{'loss': 0.1514, 'grad_norm': 11.388445854187012, 'learning_rate': 2.506265664160401e-06, 'epoch': 9.24}
{'loss': 0.1264, 'grad_norm': 7.4210100173950195, 'learning_rate': 2.4227234753550544e-06, 'epoch': 9.27}
{'loss': 0.1318, 'grad_norm': 7.7617082595825195, 'learning_rate': 2.3391812865497075e-06, 'epoch': 9.29}
{'loss': 0.1473, 'grad_norm': 6.22797155380249, 'learning_rate': 2.255639097744361e-06, 'epoch': 9.32}
{'loss': 0.1206, 'grad_norm': 5.586062908172607, 'learning_rate': 2.172096908939014e-06, 'epoch': 9.34}
{'loss': 0.1474, 'grad_norm': 20.117097854614258, 'learning_rate': 2.0885547201336676e-06, 'epoch': 9.37}
{'loss': 0.1296, 'grad_norm': 4.864410400390625, 'learning_rate': 2.0050125313283207e-06, 'epoch': 9.39}
{'loss': 0.1183, 'grad_norm': 21.537715911865234, 'learning_rate': 1.921470342522974e-06, 'epoch': 9.42}
{'loss': 0.1395, 'grad_norm': 4.98659610748291, 'learning_rate': 1.8379281537176272e-06, 'epoch': 9.44}
{'loss': 0.1235, 'grad_norm': 8.384865760803223, 'learning_rate': 1.7543859649122807e-06, 'epoch': 9.47}
{'loss': 0.1534, 'grad_norm': 10.999909400939941, 'learning_rate': 1.670843776106934e-06, 'epoch': 9.49}
{'loss': 0.1335, 'grad_norm': 6.491465091705322, 'learning_rate': 1.5873015873015873e-06, 'epoch': 9.52}
{'loss': 0.1471, 'grad_norm': 6.08101749420166, 'learning_rate': 1.5037593984962406e-06, 'epoch': 9.54}
{'loss': 0.1156, 'grad_norm': 7.777099132537842, 'learning_rate': 1.4202172096908939e-06, 'epoch': 9.57}
{'loss': 0.1456, 'grad_norm': 10.666170120239258, 'learning_rate': 1.3366750208855472e-06, 'epoch': 9.59}
{'loss': 0.1088, 'grad_norm': 5.53985071182251, 'learning_rate': 1.2531328320802005e-06, 'epoch': 9.62}
{'loss': 0.1555, 'grad_norm': 4.444268703460693, 'learning_rate': 1.1695906432748538e-06, 'epoch': 9.64}
{'loss': 0.1894, 'grad_norm': 2.7951173782348633, 'learning_rate': 1.086048454469507e-06, 'epoch': 9.67}
{'loss': 0.1245, 'grad_norm': 50.77102279663086, 'learning_rate': 1.0025062656641603e-06, 'epoch': 9.69}
{'loss': 0.1184, 'grad_norm': 9.79514217376709, 'learning_rate': 9.189640768588136e-07, 'epoch': 9.72}
{'loss': 0.1231, 'grad_norm': 14.504884719848633, 'learning_rate': 8.35421888053467e-07, 'epoch': 9.74}
{'loss': 0.1195, 'grad_norm': 6.517986297607422, 'learning_rate': 7.518796992481203e-07, 'epoch': 9.77}
{'loss': 0.1528, 'grad_norm': 13.124926567077637, 'learning_rate': 6.683375104427736e-07, 'epoch': 9.79}
{'loss': 0.1608, 'grad_norm': 5.084670066833496, 'learning_rate': 5.847953216374269e-07, 'epoch': 9.82}
{'loss': 0.1139, 'grad_norm': 3.687969923019409, 'learning_rate': 5.012531328320802e-07, 'epoch': 9.84}
{'loss': 0.135, 'grad_norm': 2.6893727779388428, 'learning_rate': 4.177109440267335e-07, 'epoch': 9.87}
{'loss': 0.1592, 'grad_norm': 6.811862468719482, 'learning_rate': 3.341687552213868e-07, 'epoch': 9.89}
{'loss': 0.1246, 'grad_norm': 8.717400550842285, 'learning_rate': 2.506265664160401e-07, 'epoch': 9.92}
{'loss': 0.1501, 'grad_norm': 7.824330806732178, 'learning_rate': 1.670843776106934e-07, 'epoch': 9.94}
{'loss': 0.126, 'grad_norm': 7.346182823181152, 'learning_rate': 8.35421888053467e-08, 'epoch': 9.97}
{'loss': 0.151, 'grad_norm': 23.138254165649414, 'learning_rate': 0.0, 'epoch': 9.99}
{'eval_loss': 0.07526067644357681, 'eval_accuracy': 0.9833799088101192, 'eval_runtime': 27.8619, 'eval_samples_per_second': 244.025, 'eval_steps_per_second': 7.645, 'epoch': 9.99}
{'train_runtime': 6837.8875, 'train_samples_per_second': 74.72, 'train_steps_per_second': 0.584, 'train_loss': 0.4670510420823157, 'epoch': 9.99}
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3990/3990 [1:53:57<00:00,  1.71s/it]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, \
    Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict, concatenate_datasets

import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'

def load_text_file(file_path, max_len=300_000):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    arr = [line.strip() for line in lines]
    return arr if len(arr) < max_len else arr[:max_len]


def prepare_dataset():
    file_paths = {
        'en_ko': 'en_ko.txt',
        'ko_en': 'ko_en.txt',
        'en_vi': 'en_vi.txt',
        'vi_en': 'vi_en.txt',
        'vi_ko': 'vi_ko.txt',
        'ko_vi': 'ko_vi.txt',
    }

    datasets = {}
    source_path = ""
    target_path = ""
    pre_key = ""
    for id, (key, path) in enumerate(file_paths.items()):
        if id % 2 == 0:
            source_path = path
            pre_key = key
        else:
            target_path = path
            print(source_path, target_path)
            print(pre_key, key)
            source, target = load_text_file(source_path), load_text_file(target_path)

            datasets[pre_key] = Dataset.from_dict({
                'src_lang': [pre_key.split('_')[0]] * len(source),
                'tgt_lang': [pre_key.split('_')[1]] * len(source),
                'source_texts': source,
                'target_texts': target
            })

            datasets[key] = Dataset.from_dict({
                'src_lang': [key.split('_')[0]] * len(source),
                'tgt_lang': [key.split('_')[1]] * len(source),
                'source_texts': target,
                'target_texts': source
            })

    dataset_dict = DatasetDict({
        'train': concatenate_datasets([datasets['en_ko'].train_test_split(test_size=0.05)['train'],
        datasets['ko_en'].train_test_split(test_size=0.05)['train'],
        datasets['en_vi'].train_test_split(test_size=0.05)['train'],
        datasets['vi_en'].train_test_split(test_size=0.05)['train'],
        datasets['vi_ko'].train_test_split(test_size=0.05)['train'],
        datasets['ko_vi'].train_test_split(test_size=0.05)['train']]),

        'test': concatenate_datasets([datasets['en_ko'].train_test_split(test_size=0.05)['test'],
        datasets['ko_en'].train_test_split(test_size=0.05)['test'],
        datasets['en_vi'].train_test_split(test_size=0.05)['test'],
        datasets['vi_en'].train_test_split(test_size=0.05)['test'],
        datasets['vi_ko'].train_test_split(test_size=0.05)['test'],
        datasets['ko_vi'].train_test_split(test_size=0.05)['test']])
    })
    dataset_dict = dataset_dict.shuffle(seed=23)
    print("shuffled..")
    return dataset_dict


def preprocess_function(examples, tokenizer):
    tokenizer.src_lang = examples['src_lang']
    tokenizer.tgt_lang = examples['tgt_lang']

    source_texts = examples['source_texts']
    target_texts = examples['target_texts']

    model_inputs = tokenizer(source_texts, truncation=True, padding=True, max_length=256)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, truncation=True, padding=True, max_length=256)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


def tokenize_dataset(dataset, tokenizer):
    tokenized_datasets = {}

    # tokenized_datasets['train'] = dataset['train'].map(
    #     lambda examples: preprocess_function(examples, tokenizer),remove_columns=dataset["train"].column_names)
    # tokenized_datasets['test'] = dataset['test'].map(
    #     lambda examples: preprocess_function(examples, tokenizer),remove_columns=dataset["train"].column_names)

    tokenized_datasets = dataset.map(
        lambda examples: preprocess_function(examples, tokenizer), remove_columns=dataset["train"].column_names)

    return tokenized_datasets


# Fine-tune the model
def fine_tune_model(tokenized_dataset, tokenizer):
    training_args = Seq2SeqTrainingArguments(
        output_dir="./m2m100-finetuned",
        # evaluation_strategy="epoch",
        evaluation_strategy=None,
        # save_strategy="steps",
        save_steps=5000,
        learning_rate=3e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=5,
        num_train_epochs=30,
        predict_with_generate=True,
        remove_unused_columns=False,
        # fp16=True,
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()

    return trainer

# Main function to run the process
if __name__ == "__main__":
    model_name = "facebook/m2m100_418M"
    model = M2M100ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = M2M100Tokenizer.from_pretrained(model_name)

    # Step 1: Prepare the dataset
    dataset = prepare_dataset()

    # Step 2: Tokenize the dataset
    tokenized_dataset = tokenize_dataset(dataset, tokenizer)

    # Step 3: Fine-tune the model
    trainer = fine_tune_model(tokenized_dataset, tokenizer)

    # Step 4: Save the fine-tuned model
    model.save_pretrained("path_to_save_finetuned_model")
    tokenizer.save_pretrained("path_to_save_finetuned_tokenizer")
